In [1]:
import pandas as pd
import requests
from keys import KEY_TWO

In [2]:
# Configure URL for pd.read_csv
# Full sheet URL == https://docs.google.com/spreadsheets/d/1xw7y9yawF6i35BTfP9M1uUawJvwpacz01Xq4MEZszBs/edit#gid=0
workbook_id = "1xw7y9yawF6i35BTfP9M1uUawJvwpacz01Xq4MEZszBs"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{workbook_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [3]:
year_df = pd.read_csv(url, parse_dates=['Release_Date'])
year_df

,Movie_Title,Release_Date,Tomatometer
0,Episode IV – A New Hope,1977-05-25,92%
1,Episode V – The Empire Strikes Back,1980-05-21,94%
2,Episode VI – Return of the Jedi,1983-05-25,82%
3,Episode I – The Phantom Menace,1999-05-19,52%
4,Episode II – Attack of the Clones,2002-05-16,65%
5,Episode III – Revenge of the Sith,2005-05-19,80%
6,Episode VII – The Force Awakens,2015-12-18,93%
7,Episode VIII – The Last Jedi,2017-12-15,91%
8,Episode IX – The Rise of Skywalker,2019-12-20,52%
9,Star Wars: The Clone Wars,2008-08-15,18%


In [4]:
year_df['Release_Date'] = year_df['Release_Date'].dt.strftime('%Y')
year_df

,Movie_Title,Release_Date,Tomatometer
0,Episode IV – A New Hope,1977,92%
1,Episode V – The Empire Strikes Back,1980,94%
2,Episode VI – Return of the Jedi,1983,82%
3,Episode I – The Phantom Menace,1999,52%
4,Episode II – Attack of the Clones,2002,65%
5,Episode III – Revenge of the Sith,2005,80%
6,Episode VII – The Force Awakens,2015,93%
7,Episode VIII – The Last Jedi,2017,91%
8,Episode IX – The Rise of Skywalker,2019,52%
9,Star Wars: The Clone Wars,2008,18%


In [5]:

# mask = year_df['Movie_Title'].str.contains(' – ')

for item, str in year_df['Movie_Title'].items():
        head, sep, tail = str.partition(' – ')
        year_df['Movie_Title'].replace(to_replace=str, value = head, inplace=True)
year_df



        



,Movie_Title,Release_Date,Tomatometer
0,Episode IV,1977,92%
1,Episode V,1980,94%
2,Episode VI,1983,82%
3,Episode I,1999,52%
4,Episode II,2002,65%
5,Episode III,2005,80%
6,Episode VII,2015,93%
7,Episode VIII,2017,91%
8,Episode IX,2019,52%
9,Star Wars: The Clone Wars,2008,18%


In [6]:
parameters = {'theme' : 'Star Wars', 'pageSize' : 900}
sw_set_list = requests.get(f"https://brickset.com/api/v3.asmx/getSets?apiKey={KEY_TWO}&userHash=&params={parameters}")
sw_data = sw_set_list.json()
sw_df = pd.json_normalize(sw_data,'sets')
sw_df.columns


Index(['setID', 'number', 'numberVariant', 'name', 'year', 'theme',
       'themeGroup', 'subtheme', 'category', 'released', 'pieces',
       'bricksetURL', 'rating', 'reviewCount', 'packagingType', 'availability',
       'instructionsCount', 'additionalImageCount', 'lastUpdated',
       'image.thumbnailURL', 'image.imageURL', 'collections.ownedBy',
       'collections.wantedBy', 'dimensions.height', 'dimensions.width',
       'dimensions.depth', 'barcode.EAN', 'barcode.UPC', 'minifigs',
       'dimensions.weight', 'LEGOCom.US.retailPrice',
       'LEGOCom.US.dateFirstAvailable', 'LEGOCom.US.dateLastAvailable',
       'LEGOCom.UK.retailPrice', 'LEGOCom.UK.dateFirstAvailable',
       'LEGOCom.UK.dateLastAvailable', 'ageRange.min', 'ageRange.max',
       'LEGOCom.CA.retailPrice', 'LEGOCom.CA.dateFirstAvailable',
       'LEGOCom.CA.dateLastAvailable', 'LEGOCom.DE.retailPrice',
       'LEGOCom.DE.dateFirstAvailable', 'LEGOCom.DE.dateLastAvailable'],
      dtype='object')

In [7]:
drop_list = [
            'numberVariant',
            'released',
            'category',
            'bricksetURL',
            'reviewCount',
            'packagingType',
            'availability',
            'instructionsCount',
            'additionalImageCount',
            'lastUpdated',
            'image.thumbnailURL',
            'image.imageURL',
            'collections.ownedBy',
            'collections.wantedBy',
            'dimensions.height',
            'dimensions.width',
            'dimensions.depth',
            'LEGOCom.US.retailPrice',
            'LEGOCom.US.dateFirstAvailable',
            'LEGOCom.US.dateLastAvailable',
            'LEGOCom.UK.retailPrice',
            'LEGOCom.UK.dateFirstAvailable',
            'LEGOCom.UK.dateLastAvailable',
            'LEGOCom.CA.retailPrice',
            'LEGOCom.CA.dateFirstAvailable',
            'LEGOCom.CA.dateLastAvailable',
            'dimensions.weight',
            'barcode.EAN',
            'barcode.UPC',
            'minifigs',
            'LEGOCom.DE.retailPrice',
            'LEGOCom.DE.dateFirstAvailable',
            'LEGOCom.DE.dateLastAvailable'
            ]
sw_df.drop(columns=drop_list, inplace=True)
sw_df['subtheme'].replace(to_replace={'The Clone Wars' : 'Star Wars: The Clone Wars', 
                                      'The Force Awakens' : 'Episode IX', 
                                      'The Last Jedi' : 'Episode X', 
                                      'The Rise of Skywalker' : 'Episode XI' }, inplace=True)

In [8]:
mask_two = sw_df[sw_df['rating'] == 0].index
sw_df.drop(mask_two, inplace=True)
pieces_null = sw_df['pieces'].isnull().values.any()
print(pieces_null)
if pieces_null == True:
    sw_df.dropna(subset=['pieces'], inplace=True)

sw_df['pieces'] = sw_df['pieces'].astype(pd.Int64Dtype())
sw_df['ageRange.min'] = sw_df['ageRange.min'].astype(pd.Int64Dtype())
sw_df['ageRange.max'] = sw_df['ageRange.max'].astype(pd.Int64Dtype())
sw_df.shape

False


(729, 11)

In [9]:
sw_df.columns
sw_df['subtheme'].value_counts()


Star Wars: The Clone Wars    78
Magazine Gift                72
Episode IV                   49
Episode III                  48
Episode V                    43
Episode VI                   41
Episode I                    40
Ultimate Collector Series    35
Buildable Figures            29
Episode IX                   26
Microfighters                24
Promotional                  21
Mini Building Set            21
Episode II                   20
Episode X                    17
Rebels                       17
Planet Set                   12
Rogue One                    12
Solo                         12
Minifig Pack                 11
Episode XI                   11
Seasonal                     11
Legends                      11
MicroFighters                11
The Mandalorian              10
Technic                       9
Original Content              7
Helmet Collection             7
The Old Republic              5
Battlefront                   4
Diorama Collection            3
Master B

In [13]:
sw_df.groupby(['subtheme']).mean()

,setID,year,pieces,rating,ageRange.min,ageRange.max
subtheme,,,,,,
Battlefront,25445.250000,2016.500000,105.75,3.900000,6.0,12.0
Book Parts,26411.000000,2016.000000,73.0,3.600000,8.0,<NA>
Boost,29296.000000,2019.000000,1177.0,3.700000,8.0,<NA>
Buildable Figures,25733.344828,2016.448276,121.896552,3.872414,7.76,13.68
Diorama Collection,32125.000000,2022.000000,822.333333,4.433333,18.0,<NA>
Episode I,10649.275000,2007.300000,321.575,3.860000,7.066667,12.923077
Episode II,11164.150000,2009.200000,367.25,4.080000,7.5,12.888889
Episode III,15037.229167,2011.166667,341.9375,4.025000,7.518519,12.666667
Episode IV,15823.224490,2011.489796,451.081633,4.004082,7.8125,13.111111


In [11]:
sw_df.to_csv('./sw_set_list.csv')